In [8]:
# https://www.pyimagesearch.com/2015/05/25/basic-motion-detection-and-tracking-with-python-and-opencv/
import imutils
import argparse
import datetime
import time
import cv2

# vs = cv2.VideoCapture(0)
vs = cv2.VideoCapture("/home/andrea/Desktop/DSC_2484.MOV")
# loop over the frames of the video
firstFrame = None

length = 10 #Length, in meters, of the captured frame
pxl_width = 0
m_per_px = 0

while True:
    ret, firstFrame = vs.read()
#     print("Before:", firstFrame.shape)
    firstFrame = cv2.resize(firstFrame, (1280, 720))
    if (pxl_width == 0):
        pxl_width = firstFrame.shape[0]
        m_per_px = length/pxl_width
        
#     print("After:", firstFrame.size())

#     if firstFrame is None:
#         continue
    firstFrame = cv2.cvtColor(firstFrame, cv2.COLOR_BGR2GRAY)
    firstFrame = cv2.GaussianBlur(firstFrame, (21, 21), 0)
    cv2.imshow("First frame", firstFrame)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key is pressed, break from the loop
    if key == ord("q"):
        break

print("Length", length)
print("pxl_width", pxl_width)
print("m_per_px", m_per_px)

        
while True:
    cv2.imshow("First frame", firstFrame)
    
    # grab the current frame and initialize the occupied/unoccupied
    # text
    ret, frame = vs.read()
    if not ret:
        break
    frame = cv2.resize(frame, (1280, 720))
#     frame = frame if args.get("video", None) is None else frame[1]
    text = "Unoccupied"
    # if the frame could not be grabbed, then we have reached the end
    # of the video
    if frame is None:
        print('Sheeesh')
        continue
    # resize the frame, convert it to grayscale, and blur it
#     frame = cv2.resize(frame, width=500)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
    # if the first frame is None, initialize it
    if firstFrame is None:
        firstFrame = gray
        continue
    # compute the absolute difference between the current frame and
    # first frame
    frameDelta = cv2.absdiff(firstFrame, gray)
#     frameDelta = cv2.multiply(frameDelta, frameDelta)
#     frameDelta = frameDelta**2
    thresh = cv2.threshold(frameDelta, 50, 255, cv2.THRESH_BINARY)[1]
    # dilate the thresholded image to fill in holes, then find contours
    # on thresholded image
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    # loop over the contours
    for c in cnts:
        # if the contour is too small, ignore it
#         if cv2.contourArea(c) < args["min_area"]:
#             continue
        if cv2.contourArea(c) < 300:
            continue
        # compute the bounding box for the contour, draw it on the frame,
        # and update the text
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        text = "Occupied"
        
    # draw the text and timestamp on the frame
    cv2.putText(frame, "Room Status: {}".format(text), (10, 20),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.putText(frame, datetime.datetime.now().strftime("%A %d %B %Y %I:%M:%S%p"),
        (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)
    # show the frame and record if the user presses a key
    cv2.imshow("Security Feed", frame)
    cv2.imshow("Thresh", thresh)
    cv2.imshow("Frame Delta", frameDelta)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key is pressed, break from the loop
    if key == ord("q"):
        break
# cleanup the camera and close any open windows
# vs.stop() if args.get("video", None) is None else vs.release()
vs.release()
cv2.destroyAllWindows()

Length 10
pxl_width 720
m_per_px 0.013888888888888888
